  <img width="800px" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/smart-claims/rule_engine.png" />

In [0]:
%sql
CREATE OR REPLACE TABLE smart_claims_dev.03_gold.customer_claim_policy_telematics_predicted AS 
  SELECT
    t.*,
    a.* EXCEPT (chassis_no, claim_no)
  FROM
    smart_claims_dev.03_gold.customer_claim_policy_telematics t
    JOIN smart_claims_dev.03_gold.claim_images_predicted a USING(claim_no)

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE smart_claims_dev.02_silver.claims_rules (
  rule_id BIGINT GENERATED ALWAYS AS IDENTITY,
  rule STRING, 
  check_name STRING,
  check_code STRING,
  check_severity STRING,
  is_active Boolean
);

In [0]:
def insert_rule(rule, name, code, severity, is_active):
  spark.sql(f"INSERT INTO smart_claims_dev.02_silver.claims_rules(rule,check_name, check_code, check_severity,is_active) values('{rule}', '{name}', '{code}', '{severity}', {is_active})")

In [0]:
invalid_policy_date = '''
  CASE WHEN to_date(pol_eff_date, "yyyy-MM-dd") < to_date(claim_date) and to_date(pol_expiry_date, "yyyy-MM-dd") < to_date(claim_date) THEN "VALID" 
  ELSE "NOT VALID"  
  END
'''

insert_rule('invalid policy date', 'valid_date', invalid_policy_date, 'HIGH', True)

In [0]:
exceeds_policy_amount = '''
CASE WHEN  sum_insured >= total 
    THEN "claim value in the range of premium"
    ELSE "claim value more than premium"
END 
'''
insert_rule('exceeds policy amount', 'valid_amount', exceeds_policy_amount, 'HIGH', True)

In [0]:
severity_mismatch = '''
CASE WHEN    incident_severity="Total Loss" AND damage_prediction.label = "major" THEN  "Severity matches the report"
       WHEN  incident_severity="Major Damage" AND damage_prediction.label = "minor"  THEN  "Severity matches the report"
       WHEN  (incident_severity="Minor Damage" OR incident_severity="Trivial Damage") AND damage_prediction.label = "ok" THEN  "Severity matches the report"
       ELSE "Severity does not match"
END 
'''

insert_rule('severity mismatch', 'reported_severity_check', severity_mismatch, 'HIGH', True)

In [0]:
exceeds_speed = '''
CASE WHEN  telematics_speed <= 45 and telematics_speed > 0 THEN  "Normal Speed"
       WHEN telematics_speed > 45 THEN  "High Speed"
       ELSE "Invalid speed"
END
'''
insert_rule('exceeds speed', 'speed_check', exceeds_speed, 'HIGH', True)

In [0]:
release_funds = '''
CASE WHEN  reported_severity_check="Severity matches the report" and valid_amount="claim value in the range of premium" and valid_date="VALID" then "release funds"
       ELSE "claim needs more investigation" 
END
'''
insert_rule('release funds', 'fund_release', release_funds, 'HIGH', True)

In [0]:
from pyspark.sql.functions import expr

df = spark.sql("SELECT * FROM smart_claims_dev.03_gold.customer_claim_policy_telematics_predicted")
rules = spark.sql('SELECT * FROM smart_claims_dev.02_silver.claims_rules where is_active=true order by rule_id').collect()
for rule in rules:
  print(rule.rule, rule.check_code)
  df=df.withColumn(rule.check_name, expr(rule.check_code))

#overwrite table with new insights
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("smart_claims_dev.03_gold.claim_insights")

invalid policy date 
  CASE WHEN to_date(pol_eff_date, "yyyy-MM-dd") < to_date(claim_date) and to_date(pol_expiry_date, "yyyy-MM-dd") < to_date(claim_date) THEN "VALID" 
  ELSE "NOT VALID"  
  END

exceeds policy amount 
CASE WHEN  sum_insured >= total 
    THEN "claim value in the range of premium"
    ELSE "claim value more than premium"
END 

severity mismatch 
CASE WHEN    incident_severity="Total Loss" AND damage_prediction.label = "major" THEN  "Severity matches the report"
       WHEN  incident_severity="Major Damage" AND damage_prediction.label = "minor"  THEN  "Severity matches the report"
       WHEN  (incident_severity="Minor Damage" OR incident_severity="Trivial Damage") AND damage_prediction.label = "ok" THEN  "Severity matches the report"
       ELSE "Severity does not match"
END 

exceeds speed 
CASE WHEN  telematics_speed <= 45 and telematics_speed > 0 THEN  "Normal Speed"
       WHEN telematics_speed > 45 THEN  "High Speed"
       ELSE "Invalid speed"
END

release fund

In [0]:
display(rules)

rule_id,rule,check_name,check_code,check_severity,is_active
1,exceeds policy amount,valid_amount,"CASE WHEN sum_insured >= total THEN ""claim value in the range of premium"" ELSE ""claim value more than premium"" END",HIGH,true
2,severity mismatch,reported_severity_check,"CASE WHEN incident_severity=""Total Loss"" AND damage_prediction.label = ""major"" THEN ""Severity matches the report"" WHEN incident_severity=""Major Damage"" AND damage_prediction.label = ""minor"" THEN ""Severity matches the report"" WHEN (incident_severity=""Minor Damage"" OR incident_severity=""Trivial Damage"") AND damage_prediction.label = ""ok"" THEN ""Severity matches the report"" ELSE ""Severity does not match"" END",HIGH,true
3,exceeds speed,speed_check,"CASE WHEN telematics_speed <= 45 and telematics_speed > 0 THEN ""Normal Speed"" WHEN telematics_speed > 45 THEN ""High Speed"" ELSE ""Invalid speed"" END",HIGH,true
4,release funds,fund_release,"CASE WHEN reported_severity_check=""Severity matches the report"" and valid_amount=""claim value in the range of premium"" and valid_date=""VALID"" then ""release funds"" ELSE ""claim needs more investigation"" END",HIGH,true
5,invalid policy date,valid_date,"CASE WHEN to_date(pol_eff_date, ""yyyy-MM-dd"") < to_date(claim_date) and to_date(pol_expiry_date, ""yyyy-MM-dd"") < to_date(claim_date) THEN ""VALID"" ELSE ""NOT VALID"" END",HIGH,true
6,invalid policy date,valid_date,"CASE WHEN to_date(pol_eff_date, ""yyyy-MM-dd"") < to_date(claim_date) and to_date(pol_expiry_date, ""yyyy-MM-dd"") < to_date(claim_date) THEN ""VALID"" ELSE ""NOT VALID"" END",HIGH,true


In [0]:
%sql
SELECT *  FROM smart_claims_dev.03_gold.claim_insights
LIMIT 1

claim_no,chassis_no,policy_no,claim_date,months_as_customer,injury,property,vehicle,total,collision_type,number_of_vehicles_involved,driver_age,insured_relationship,license_issue_date,incident_date,incident_hour,incident_type,incident_severity,number_of_witnesses,suspicious_activity,cust_id,policytype,pol_issue_date,pol_eff_date,pol_expiry_date,make,model,model_year,use_of_vehicle,product,sum_insured,premium,deductable,customer_id,date_of_birth,borough,neighborhood,zip_code,firstname,lastname,address,lat_long,telematics_speed,telematics_latitude,telematics_longitude,image_name,path,modificationTime,length,content,damage_prediction,image_id,_rescued_data,valid_date,valid_amount,reported_severity_check,speed_check,fund_release
1f765afc-5d41-4ef2-87f7-a60a2aeaf92c,LVVDC21B3GD020661,102144371,2020-06-16,298,5830,11660,40810,58300,Rear Collision,1,27.0,not-in-family,2012-04-30,2020-10-04,3,Single Vehicle Collision,Minor Damage,1,false,3577.0,COMP,2020-05-03,2020-06-06,2021-07-05,CHERY,ARIZO 7,2016,PRIVATE,STANDARD,17203.0,1030.0,500,3577,2006-05-16,QUEENS,WEST QUEENS,11377,Julius,Wagner,"QUEENS, 11377","List(-52.034763, 124.71737)",31.039113200704257,40.76418333333333,-73.88939078333333,2_High.jpg,dbfs:/Volumes/smart_claims_dev/00_landing/claims/images/2_High.jpg,2025-08-10T08:58:30Z,1928661,"List(iVBORw0KGgoAAAANSUhEUgAABAAAAAQACAIAAADwf7zUAAAAemVYSWZNTQAqAAAACAACknwAAgAAAD0AAAAmkoYAAgAAABYAAABkAAAAAE9wZW5BSS0tSW1hZ2VHZW5lcmF0aW9uLS1nZW5lcmF0aW9uLVh0SkVrOHhpem5mZHE= (truncated), ...)","List(major, 0.4651804268360138)",4,null,NOT VALID,claim value more than premium,Severity does not match,Normal Speed,claim needs more investigation
